# Stress inversion of the Geysers focal mechanisms from NCEDC

Cite the NCEDC:
"NCEDC (2014), Northern California Earthquake Data Center. UC Berkeley Seismological Laboratory. Dataset. [doi:10.7932/NCEDC](doi:10.7932/NCEDC)."

Acknowledge the NCEDC:
"Waveform data, metadata, or data products for this study were accessed through the Northern California Earthquake Data Center (NCEDC), [doi:10.7932/NCEDC](doi:10.7932/NCEDC)."

You may have to install `mplstereonet` [https://github.com/joferkington/mplstereonet](https://github.com/joferkington/mplstereonet), and `colorcet` [https://colorcet.holoviz.org/](https://colorcet.holoviz.org/).

In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

import cartopy as ctp
import colorcet as cc
from obspy.imaging import beachball as obb

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter 

import mplstereonet as mpl

import ILSI

from time import time as give_time

# plot data in Kaverina diagram
sys.path.append(os.path.join('/home/eric/Dropbox (MIT)/DANA/temporary_working_dir/focal_mechanisms/FMC/'))
import plotFMC
import functionsFMC
from functionsFMC import kave, mecclass

# set plotting parameters
import seaborn as sns
sns.set(font_scale=1.3)
sns.set_style('ticks')
sns.set_palette('colorblind')
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['svg.fonttype'] = 'none'

# define the color palette
_colors_ = ['C0', 'C2', 'C1', 'C4']

In [ ]:
# path variables
data_path = '../data'
dataset_fn = 'Geysers_dataset.csv'
# load dataset
data = pd.read_csv(os.path.join(data_path, dataset_fn), sep='\t', index_col=0, header=0)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
print(data)

In [ ]:
def add_scale_bar(ax, x_start, y_start, distance, source_crs,
                  azimuth=90., **kwargs):
    """
    Parameters
    -----------
    ax: GeoAxes instance
        The axis on which we want to add a scale bar.
    x_start: float
        The x coordinate of the left end of the scale bar,
        given in the axis coordinate system, i.e. from 0 to 1.
    y_start: float
        The y coordinate of the left end of the scale bar,
        given in the axis coordinate system, i.e. from 0 to 1.
    distance: float
        The distance covered by the scale bar, in km.
    source_crs: cartopy.crs
        The coordinate system in which the data are written.
    """
    from cartopy.geodesic import Geodesic
    G = Geodesic()

    # default values
    kwargs['lw'] = kwargs.get('lw', 2)
    kwargs['color'] = kwargs.get('color', 'k')

    data_coords = ctp.crs.PlateCarree()
    # transform the axis coordinates into display coordinates
    display = ax.transAxes.transform([x_start, y_start])
    # take display coordinates into data coordinates
    data = ax.transData.inverted().transform(display)
    # take data coordinates into lon/lat
    lon_start, lat_start = data_coords.transform_point(
            data[0], data[1], source_crs)
    # get the coordinates of the end of the scale bar
    lon_end, lat_end, _ = np.asarray(G.direct(
            [lon_start, lat_start], azimuth, 1000.*distance))[0]
    ax.plot([lon_start, lon_end], [lat_start, lat_end],
            transform=data_coords, **kwargs)
    ax.text((lon_start+lon_end)/2., (lat_start+lat_end)/2.-0.001,
            '{:.0f}km'.format(distance), transform=data_coords,
            ha='center', va='top')
    return

def plot_Kaverina(strikes, dips, rakes, ax=None):
    n_earthquakes = len(strikes)
    # determine the t and p axes
    P_axis = np.zeros((n_earthquakes, 2), dtype=np.float32)
    T_axis = np.zeros((n_earthquakes, 2), dtype=np.float32)
    B_axis = np.zeros((n_earthquakes, 2), dtype=np.float32)
    faulting_type = np.zeros(n_earthquakes, dtype=np.int32)
    fm_type = []
    r2d = 180./np.pi
    for t in range(n_earthquakes):
        # first, get normal and slip vectors from
        # strike, dip, rake
        normal, slip = ILSI.utils_stress.normal_slip_vectors(strikes[t], dips[t], rakes[t])
        # second, get the t and p vectors
        p_axis, t_axis, b_axis =\
                ILSI.utils_stress.p_t_b_axes(normal, slip)
        p_bearing, p_plunge = ILSI.utils_stress.get_bearing_plunge(p_axis)
        t_bearing, t_plunge = ILSI.utils_stress.get_bearing_plunge(t_axis)
        b_bearing, b_plunge = ILSI.utils_stress.get_bearing_plunge(b_axis)
        P_axis[t, :] = p_bearing, p_plunge
        T_axis[t, :] = t_bearing, t_plunge
        B_axis[t, :] = b_bearing, b_plunge
        fm_type.append(mecclass(t_plunge, b_plunge, p_plunge))
    # get the x, y coordinates for FMC's plots
    x_kave, y_kave = kave(T_axis[:, 1], B_axis[:, 1], P_axis[:, 1])

    colors_fm = {}
    colors_fm['SS'] = 'C0'
    colors_fm['SS-N'] = 'C0'
    colors_fm['SS-R'] = 'C0'
    colors_fm['R'] = 'C1'
    colors_fm['R-SS'] = 'C1'
    colors_fm['N'] = 'C2'
    colors_fm['N-SS'] = 'C2'
    colors = np.asarray([colors_fm[fm] for fm in fm_type])

    if ax is None:
        fig = plt.figure('Kaverina_diagram', figsize=(12, 12))
        ax = fig.add_subplot(111)
    fig = plotFMC.baseplot(10, '', ax=ax)
    ax.scatter(x_kave, y_kave, color=colors)
    return fig

In [ ]:
# define region extent
lat_min_box, lat_max_box = 38.83, 38.85
lon_min_box, lon_max_box = -122.84, -122.82
cNorm_box = Normalize(vmin=np.floor(data['depth'].min()),
                      vmax=np.ceil(data['depth'].max()))
scalar_map_box = ScalarMappable(norm=cNorm_box, cmap=cc.cm.fire_r)
# define projections
data_coords = ctp.crs.PlateCarree()
projection = ctp.crs.Mercator(
    central_longitude=(lon_min_box+lon_max_box)/2.,
    min_latitude=lat_min_box, max_latitude=lat_max_box)

In [ ]:
# plot the focal mechanisms
fig_fm = plt.figure('earthquakes_Geysers', figsize=(20, 15))
gs = fig_fm.add_gridspec(2, 5)
ax_fm = fig_fm.add_subplot(gs[:, :3], projection=projection)
ax_fm.set_rasterization_zorder(1.5)
ax_fm.set_extent([lon_min_box, lon_max_box, lat_min_box, lat_max_box], crs=data_coords)
ax_fm.set_title('Geysers', pad=15)
# add meridians and latitudes
gl = ax_fm.gridlines(
    draw_labels=True, linewidth=1, alpha=0.5,
    color='k', linestyle='--')
gl.right_labels = False
gl.left_labels = True
gl.top_labels = False
gl.bottom_labels = True
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()
# add coastlines
ax_fm.add_feature(ctp.feature.GSHHSFeature(scale='full', levels=[1], rasterized=True))
# add borders
ax_fm.add_feature(ctp.feature.BORDERS)
for i in range(len(data)):
    # fetch focal mechanism
    strike, dip, rake = data.iloc[i]['strike'], data.iloc[i]['dip'], data.iloc[i]['rake']
    # determine coordinates in the axis frame of reference
    x, y = ax_fm.projection.transform_point(
        data.iloc[i]['lon'], data.iloc[i]['lat'], data_coords)
    fc = scalar_map_box.to_rgba(data.iloc[i]['depth'])
    # ---------------------------
    # uncomment if you are not planning on saving svg figures:
    bb = obb.beach(
       [strike, dip, rake], xy=(x, y), width=20,
       facecolor=fc, linewidth=0.4, axes=ax_fm)
    # ---------------------------
    # uncomment if you are planning on making svg figures
    # bb = obb.beach(
    #     [strike, dip, rake], xy=(x, y), width=100,
    #     facecolor=fc, linewidth=0.4, zorder=1)
    ax_fm.add_collection(bb)
# add colorbar
divider = make_axes_locatable(ax_fm)
cax = divider.append_axes('right', size='5%', pad=0.08, axes_class=plt.Axes)
plt.colorbar(scalar_map_box, cax=cax, label='Depth (km)', orientation='vertical')
# add scale bar 
add_scale_bar(ax_fm, 0.05, 0.1, 1., projection)
# ---------------------------------------------------
#        plot data points in P/T space
ax_pt = fig_fm.add_subplot(gs[0, 3:], projection='stereonet')
# fetch focal mechanisms
strikes, dips, rakes = data['strike'], data['dip'], data['rake']
n, s = ILSI.utils_stress.normal_slip_vectors(strikes, dips, rakes)
# compute the bearing and plunge of each P/T vector
p_or = np.zeros((len(strikes), 2), dtype=np.float32)
t_or = np.zeros((len(strikes), 2), dtype=np.float32)
for i in range(len(strikes)):
    p, t, b = ILSI.utils_stress.p_t_b_axes(n[:, i], s[:, i])
    p_or[i, :] = ILSI.utils_stress.get_bearing_plunge(p)
    t_or[i, :] = ILSI.utils_stress.get_bearing_plunge(t)
ax_pt.line(t_or[:, 1], t_or[:, 0], marker='v', color='C0')
ax_pt.line(p_or[:, 1], p_or[:, 0], marker='o', color='C3')
ax_pt.line(t_or[0, 1], t_or[0, 0], marker='v', color='C0', label='T-axis')
ax_pt.line(p_or[0, 1], p_or[0, 0], marker='o', color='C3', label='P-axis')
ax_pt.grid(True)
ax_pt.legend(loc='lower left', bbox_to_anchor=(0.90, 0.92))
# ---------------------------------------------------
#            plot data points in Kaverina diagram
ax_Kav = fig_fm.add_subplot(gs[1, 3:])
fig_fm = plot_Kaverina(strikes.values, dips.values, rakes.values, ax=ax_Kav)
plt.subplots_adjust(wspace=0.35)

## Stress tensor inversion

In [ ]:
# --------------------------------
# prepare data
# --------------------------------
strikes, dips, rakes = data['strike'], data['dip'], data['rake']
strikes_1, dips_1, rakes_1 = strikes.values, dips.values, rakes.values
rakes_1 = np.float32(rakes_1)%360.
n_earthquakes = len(strikes_1)
strikes_2, dips_2, rakes_2 = \
         np.asarray(list(map(ILSI.utils_stress.aux_plane, strikes_1, dips_1, rakes_1))).T

In [ ]:
# --------------------------------
# inversion parameters
# --------------------------------
friction_min = 0.1
friction_max = 1.0
friction_step = 0.05
n_random_selections = 30
n_stress_iter = 10
n_resamplings = 2000
n_averaging = 3
ILSI_kwargs = {}
ILSI_kwargs['max_n_iterations'] = 300
ILSI_kwargs['shear_update_atol'] = 1.e-5
Tarantola_kwargs = {}

In [ ]:
# --------------------------------
# initialize output dictionary
# --------------------------------
inversion_output = {}
methods = ['linear', 'iterative_constant_shear', 'iterative_variable_shear']
for method in methods:
    inversion_output[method] = {}
    inversion_output[method]['stress_tensor'] =\
            np.zeros((3, 3), dtype=np.float32)
    inversion_output[method]['principal_stresses'] =\
            np.zeros(3, dtype=np.float32)
    inversion_output[method]['principal_directions'] =\
            np.zeros((3, 3), dtype=np.float32)
    inversion_output[method]['misfit'] = 0.
    inversion_output[method]['boot_stress_tensor'] =\
            np.zeros((n_resamplings, 3, 3), dtype=np.float32)
    inversion_output[method]['boot_principal_stresses'] =\
            np.zeros((n_resamplings, 3), dtype=np.float32)
    inversion_output[method]['boot_principal_directions'] =\
            np.zeros((n_resamplings, 3, 3), dtype=np.float32)
    inversion_output[method]['boot_misfit'] = np.zeros(n_resamplings, dtype=np.float32)
inversion_output['iterative_constant_shear']['friction'] = 0.
inversion_output['iterative_variable_shear']['friction'] = 0.

In [ ]:
# --------------------------------
# invert the whole dataset
# --------------------------------
print(f'Linear inversion...')
# simple, linear inversion
inversion_output['linear']['stress_tensor'],\
inversion_output['linear']['principal_stresses'],\
inversion_output['linear']['principal_directions'] =\
        ILSI.ilsi.inversion_one_set(
                strikes_1, dips_1, rakes_1,
                n_random_selections=n_random_selections,
                **ILSI_kwargs,
                Tarantola_kwargs=Tarantola_kwargs,
                variable_shear=False)
print(f'Iterative constant shear inversion...')
inversion_output['iterative_constant_shear']['stress_tensor'],\
inversion_output['iterative_constant_shear']['friction'],\
inversion_output['iterative_constant_shear']['principal_stresses'],\
inversion_output['iterative_constant_shear']['principal_directions'] =\
        ILSI.ilsi.inversion_one_set_instability(
                strikes_1, dips_1, rakes_1,
                n_random_selections=n_random_selections,
                **ILSI_kwargs,
                Tarantola_kwargs=Tarantola_kwargs,
                friction_min=friction_min,
                friction_max=friction_max,
                friction_step=friction_step,
                n_stress_iter=n_stress_iter,
                n_averaging=n_averaging,
                variable_shear=False)
print(f'Iterative variable shear inversion...')
inversion_output['iterative_variable_shear']['stress_tensor'],\
inversion_output['iterative_variable_shear']['friction'],\
inversion_output['iterative_variable_shear']['principal_stresses'],\
inversion_output['iterative_variable_shear']['principal_directions'] =\
        ILSI.ilsi.inversion_one_set_instability(
                strikes_1, dips_1, rakes_1,
                n_random_selections=n_random_selections,
                **ILSI_kwargs,
                Tarantola_kwargs=Tarantola_kwargs,
                friction_min=friction_min,
                friction_max=friction_max,
                friction_step=friction_step,
                n_stress_iter=n_stress_iter,
                n_averaging=n_averaging,
                weighted=True,
                plot=False)
for method in methods:
    R = ILSI.utils_stress.R_(inversion_output[method]['principal_stresses'])
    I, fp_strikes, fp_dips, fp_rakes = ILSI.ilsi.compute_instability_parameter(
            inversion_output[method]['principal_directions'], R, 0.6,
            strikes_1, dips_1, rakes_1, strikes_2, dips_2, rakes_2,
            return_fault_planes=True)
    inversion_output[method]['misfit'] =\
            np.mean(ILSI.utils_stress.mean_angular_residual(
                inversion_output[method]['stress_tensor'],
                fp_strikes, fp_dips, fp_rakes))

In [ ]:
# --------------------------------
# bootstrap the dataset to infer uncertainties
# --------------------------------
print(f'Linear inversion (bootstrapping)...')
t1 = give_time()
# simple, linear inversion
inversion_output['linear']['boot_stress_tensor'],\
inversion_output['linear']['boot_principal_stresses'],\
inversion_output['linear']['boot_principal_directions'] =\
        ILSI.ilsi.inversion_bootstrap(
                strikes_1, dips_1, rakes_1,
                n_resamplings=n_resamplings,
                Tarantola_kwargs=Tarantola_kwargs,
                variable_shear=False)
t2 = give_time()
print(f'Done in {t2-t1:.2f}sec!')
t1 = give_time()
print(f'Iterative constant shear inversion (bootstrapping)...')
inversion_output['iterative_constant_shear']['boot_stress_tensor'],\
inversion_output['iterative_constant_shear']['boot_principal_stresses'],\
inversion_output['iterative_constant_shear']['boot_principal_directions'] =\
        ILSI.ilsi.inversion_bootstrap_instability(
                inversion_output['iterative_constant_shear']['principal_directions'],
                ILSI.utils_stress.R_(inversion_output['iterative_constant_shear']['principal_stresses']),
                strikes_1, dips_1, rakes_1,
                inversion_output['iterative_constant_shear']['friction'],
                **ILSI_kwargs,
                Tarantola_kwargs=Tarantola_kwargs,
                n_stress_iter=n_stress_iter,
                n_resamplings=n_resamplings,
                parallel=True,
                variable_shear=False)
t2 = give_time()
print(f'Done in {t2-t1:.2f}sec!')
t1 = give_time()
print(f'Iterative variable shear inversion (bootstrapping)...')
inversion_output['iterative_variable_shear']['boot_stress_tensor'],\
inversion_output['iterative_variable_shear']['boot_principal_stresses'],\
inversion_output['iterative_variable_shear']['boot_principal_directions'] =\
        ILSI.ilsi.inversion_bootstrap_instability(
                inversion_output['iterative_variable_shear']['principal_directions'],
                ILSI.utils_stress.R_(inversion_output['iterative_variable_shear']['principal_stresses']),
                strikes_1, dips_1, rakes_1,
                inversion_output['iterative_variable_shear']['friction'],
                **ILSI_kwargs,
                Tarantola_kwargs=Tarantola_kwargs,
                n_stress_iter=n_stress_iter,
                n_resamplings=n_resamplings,
                parallel=True,
                variable_shear=True)
t2 = give_time()
print(f'Done in {t2-t1:.2f}sec!')
for method in methods:
    for b in range(n_resamplings):
        R = ILSI.utils_stress.R_(inversion_output[method]['boot_principal_stresses'][b, ...])
        I, fp_strikes, fp_dips, fp_rakes = ILSI.ilsi.compute_instability_parameter(
                inversion_output[method]['boot_principal_directions'][b, ...], R, 0.6,
                strikes_1, dips_1, rakes_1, strikes_2, dips_2, rakes_2,
                return_fault_planes=True)
        inversion_output[method]['boot_misfit'][b] =\
                np.mean(ILSI.utils_stress.mean_angular_residual(
                    inversion_output[method]['boot_stress_tensor'][b, ...],
                    fp_strikes, fp_dips, fp_rakes))

In [ ]:
inversion_output['strikes'] = np.stack((strikes_1, strikes_2), axis=1)
inversion_output['dips'] = np.stack((dips_1, dips_2), axis=1)
inversion_output['rakes'] = np.stack((rakes_1, rakes_2), axis=1)

In [ ]:
def plot_inverted_stress_tensors(inversion_output, axes=None, figtitle='', **kwargs):
    hist_kwargs = {}
    hist_kwargs['smoothing_sig'] = kwargs.get('smoothing_sig', 1)
    hist_kwargs['nbins'] = kwargs.get('nbins', 200)
    hist_kwargs['return_count'] = kwargs.get('return_count', True)
    hist_kwargs['confidence_intervals'] = kwargs.get('confidence_intervals', [95.])
    markers = ['o', 's', 'v']
    methods = ['linear', 'iterative_constant_shear', 'iterative_variable_shear']
    n_resamplings = inversion_output['linear']['boot_principal_directions'].shape[0]
    fig = plt.figure('inverted_stress_tensors_Geysers', figsize=(15, 9))
    fig.suptitle(figtitle)
    gs = fig.add_gridspec(nrows=3, ncols=3, top=0.88, bottom=0.11,
                           left=0.15, right=0.85, hspace=0.4, wspace=0.7)
    axes = []
    ax1 = fig.add_subplot(gs[0, 0], projection='stereonet')
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    #for j, method in enumerate(methods):
    for j in [0, 1, 2]:
        method = methods[j]
        R = ILSI.utils_stress.R_(inversion_output[method]['principal_stresses'])
        for k in range(3):
            if k == 0:
                label = '{}:\n'.format(method.replace('_', ' ').capitalize())
                for k2 in range(3):
                    az, pl = ILSI.utils_stress.get_bearing_plunge(
                            inversion_output[method]['principal_directions'][:, k2])
                    label += r'$\sigma_{{{:d}}}$: {:.1f}'u'\u00b0''|{:.1f}'u'\u00b0'', '.\
                            format(k2+1, az, pl)
                label += ' R={:.2f}, $\\beta$={:.1f}'u'\u00b0'.\
                        format(R, inversion_output[method]['misfit'])
            else:
                label = ''
            az, pl = ILSI.utils_stress.get_bearing_plunge(
                    inversion_output[method]['principal_directions'][:, k])
            ax1.line(pl, az, marker=markers[k], markeredgecolor=_colors_[j], markeredgewidth=2,
                     markerfacecolor='none', markersize=[15, 15, 15, 15][j], label=label, zorder=2)
            boot_pd_stereo = np.zeros((n_resamplings, 2), dtype=np.float32)
            for b in range(n_resamplings):
                boot_pd_stereo[b, :] = ILSI.utils_stress.get_bearing_plunge(
                        inversion_output[method]['boot_principal_directions'][b, :, k])
            count, lons_g, lats_g, levels = ILSI.utils_stress.get_CI_levels(
                    boot_pd_stereo[:, 0], boot_pd_stereo[:, 1], **hist_kwargs)
            ax1.contour(lons_g, lats_g, count, levels=levels, vmin=0.,
                        linestyles=['solid', 'dashed', 'dashdot'][k],
                        linewidths=0.75, colors=_colors_[j], zorder=1.1)
        axes.append(ax1)
        Rs = np.zeros(n_resamplings, dtype=np.float32)
        for b in range(n_resamplings):
            Rs[b] = ILSI.utils_stress.R_(inversion_output[method]['boot_principal_stresses'][b, :])
        ax2.hist(Rs, range=(0., 1.), bins=20, lw=2.5, color=_colors_[j], histtype='step')
    ax2.set_xlabel('Shape Ratio')
    ax2.set_ylabel('Count')
    # plot stress magnitudes
    #for i, method in enumerate(methods):
    for i in [0, 1, 2]:
        method = methods[i]
        pd = inversion_output[method]['principal_directions']
        R = ILSI.utils_stress.R_(inversion_output[method]['principal_stresses'])
        I, fp_strikes, fp_dips, fp_rakes = ILSI.ilsi.compute_instability_parameter(
                pd, R, 0.6,
                strikes_1, dips_1, rakes_1, strikes_2, dips_2, rakes_2,
                return_fault_planes=True)    
        normals, slips = ILSI.utils_stress.normal_slip_vectors(
                fp_strikes, fp_dips, fp_rakes)
        _, _, shear = ILSI.utils_stress.compute_traction(
                inversion_output[method]['stress_tensor'], normals.T)
        Ts_mag = np.sqrt(np.sum(shear**2, axis=-1))
        sig_Ts = np.std(Ts_mag)
        ax3.hist(Ts_mag, range=(0., 1.0),
                 bins=20, color=_colors_[i], alpha=0.5,
                 label=method.replace('_', ' ').capitalize())
    ax3.set_ylabel('Count')
    ax3.set_xlabel('Shear stress $\\vert \\tau \\vert$')
    ax3.legend(bbox_to_anchor=(-0.25, 1.05), loc='lower left')
    for ax in axes:
        #ax.grid(True)
        ax.legend(loc='upper left', bbox_to_anchor=(-0.5, -0.30))
    plt.subplots_adjust(top=0.88, bottom=0.11,
                        left=0.05, right=0.95,
                        hspace=0.4, wspace=0.4)
    return fig

In [ ]:
# plot results
kwargs = {}
kwargs['smoothing_sig'] = 5 # control the smoothness of the confidence intervals
kwargs['confidence_intervals'] = [95.]
fig_SI = plot_inverted_stress_tensors(inversion_output, figtitle='Geysers data set', **kwargs)

In [ ]:
# save the figures
# for fig in [fig_fm, fig_SI]:
#     fig.savefig(f'{fig._label}.png', format='png', bbox_inches='tight')
#     fig.savefig(f'{fig._label}.svg', format='svg', bbox_inches='tight')

## Some extra figures

In [ ]:
def plot_Mohr(inversion, plot_density=False):
    import synthetic_dataset as syndata
    strikes = inversion['strikes']
    dips = inversion['dips']
    rakes = inversion['rakes']
    fig, axes = plt.subplots(
        num=f'Mohr_space_Geysers', figsize=(20, 7), nrows=1, ncols=3)
    fig.suptitle(f'The Geysers geothermal field')
    for i, method in enumerate(['linear', 'iterative_constant_shear', 'iterative_variable_shear']):
        axes[i].set_title(method.replace('_', ' ').capitalize())
        st = inversion[method]['stress_tensor']
        p_sig, p_dir = ILSI.utils_stress.stress_tensor_eigendecomposition(st)
        I, s, d, r = ILSI.ilsi.compute_instability_parameter(
            p_dir, ILSI.utils_stress.R_(p_sig), 0.6, strikes[:, 0], dips[:, 0], rakes[:, 0],
            strikes[:, 1], dips[:, 1], rakes[:, 1], return_fault_planes=True)
        fig = syndata.plot_dataset_Mohr(st, s, d, plot_density=plot_density, ax=axes[i])
        fig = syndata.plot_dataset_Mohr(st, s, d, plot_density=plot_density, ax=axes[i])
        fig = syndata.plot_dataset_Mohr(st, s, d, plot_density=plot_density, ax=axes[i])
    for i, ax in enumerate(axes.flatten()):
        ax.text(
            -0.1, 1.05, f'({string.ascii_lowercase[i]})', transform=ax.transAxes, size=20)
    fig.tight_layout()
    return fig

In [ ]:
fig_Mohr = plot_Mohr(inversion_output, plot_density=True)

In [ ]:
def plot_instabilities(inversion, mu=0.60, Imin=0.80):
    strikes = inversion['strikes']
    dips = inversion['dips']
    rakes = inversion['rakes']
    fig, ax = plt.subplots(num='instabilities', figsize=(12, 12), nrows=1, ncols=1)
    for i, method in enumerate(['linear', 'iterative_constant_shear', 'iterative_variable_shear']):
        stress_tensor = inversion[method]['stress_tensor']
        p_sig, p_dir = ILSI.utils_stress.stress_tensor_eigendecomposition(stress_tensor)
        I, s, d, r = ILSI.ilsi.compute_instability_parameter(
            p_dir, ILSI.utils_stress.R_(p_sig), mu, strikes[:, 0],
            dips[:, 0], rakes[:, 0], strikes[:, 1], dips[:, 1], rakes[:, 1],
            return_fault_planes=True)
        ax.hist(np.max(I, axis=-1), bins=20, range=(Imin, 1.),
                color=_colors_[i], label=method.replace('_', ' ').capitalize(),
                alpha=1, histtype='step', linewidth=3.0)
    ax.set_xlabel(r'Instability $I$')
    ax.set_ylabel('Count')
    ax.legend(loc='upper left')
    return fig

In [ ]:
fig_inst = plot_instabilities(inversion_output, Imin = 0.)
#fig_inst.savefig(fig_inst._label+'_Geysers.svg', format='svg', bbox_inches='tight')